This is a trace simulator. We first start creating a MAX_TOWERS towers in a 

In [22]:
import random
import math

Constants

In [40]:
NUMBER_OF_TOWERS = 10
NUMBER_OF_USERS = 10
CYCLES = 24


In [57]:
def distance(p1, p2):
    return math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2))

def get_random_trace():
   return [random.choice([i for i in range(NUMBER_OF_TOWERS)]) for _ in range(CYCLES)]

def agregate_data(traces):
    """
        returns how many users were in each step 
    """
    output = [[0 for _ in range(CYCLES)] for _ in range(MAX_TOWERS)]
    for tower in range(NUMBER_OF_TOWERS):
        for time in range(CYCLES):
            output[tower][time] += traces[tower][time]
    return output

In [42]:
towers = []
for _ in range(NUMBER_OF_TOWERS):
    towers.append((random.random(), random.random()))

In [52]:
distances = [[distance(towers[i], towers[j]) for j in range(NUMBER_OF_TOWERS)] for i in range(NUMBER_OF_TOWERS)]


## random probability

In [59]:
random_traces = [get_random_trace() for _ in range(NUMBER_OF_USERS)]
agregated_data = agregate_data(random_traces)
agregated_data

[[1, 0, 9, 9, 8, 0, 0, 4, 4, 6, 8, 7, 2, 5, 8, 4, 9, 1, 6, 1, 3, 0, 0, 0],
 [2, 4, 9, 6, 3, 0, 5, 1, 3, 1, 5, 6, 1, 4, 5, 8, 2, 3, 1, 3, 9, 8, 7, 8],
 [8, 1, 2, 9, 6, 8, 0, 9, 3, 3, 4, 1, 0, 4, 7, 6, 2, 1, 0, 9, 8, 8, 8, 5],
 [3, 7, 0, 7, 7, 3, 9, 3, 5, 6, 2, 2, 5, 9, 0, 5, 2, 5, 9, 0, 9, 7, 6, 1],
 [9, 5, 1, 4, 5, 0, 9, 3, 5, 4, 6, 1, 5, 1, 1, 7, 7, 6, 5, 5, 0, 8, 2, 1],
 [3, 1, 2, 2, 0, 4, 6, 8, 4, 5, 2, 9, 5, 2, 2, 7, 4, 0, 7, 2, 8, 4, 3, 8],
 [6, 2, 2, 0, 4, 5, 5, 8, 9, 8, 2, 6, 2, 6, 9, 2, 8, 7, 1, 1, 9, 5, 6, 9],
 [3, 8, 4, 6, 5, 9, 6, 7, 6, 9, 5, 0, 5, 8, 3, 2, 5, 1, 8, 6, 7, 8, 0, 6],
 [6, 5, 5, 2, 0, 7, 1, 6, 8, 8, 9, 3, 2, 3, 5, 7, 9, 7, 6, 0, 3, 1, 4, 8],
 [6, 7, 9, 6, 8, 3, 7, 1, 8, 9, 4, 6, 9, 8, 2, 6, 6, 9, 3, 3, 5, 1, 1, 4]]

## weigted probability, based only on distance